In [7]:
import mlflow

In [8]:
mlflow.set_tracking_uri("http://localhost:5000")  # Set the tracking URI to your MLflow server

In [9]:
print(f"MLFLOW TRACKING URI: {mlflow.get_tracking_uri()}")

MLFLOW TRACKING URI: http://localhost:5000


In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/notebooks/03-expriments-tracking/scenarios/artifacts_local/0', creation_time=1756884772700, experiment_id='0', last_update_time=1756884772700, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score


In [12]:
mlflow.set_experiment("Scenario1")

2025/09/03 07:33:36 INFO mlflow.tracking.fluent: Experiment with name 'Scenario1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/ubuntu/notebooks/03-expriments-tracking/scenarios/artifacts_local/1', creation_time=1756884816790, experiment_id='1', last_update_time=1756884816790, lifecycle_stage='active', name='Scenario1', tags={}>

In [13]:
with mlflow.start_run():
    # Load data
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    # Predict
    mlflow.log_params(params)
    model = LogisticRegression(**params).fit(X, y)
    y_pred= model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(model, name="models")

    print(f"Logged model with accuracy: {accuracy}")

2025/09/03 07:34:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged model with accuracy: 0.96
🏃 View run skillful-dolphin-249 at: http://localhost:5000/#/experiments/1/runs/f3653b8c3a1c450c9cd4e40ba0460755
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [14]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://localhost:5000")

In [15]:
client.search_registered_models()

[]

In [24]:
run_id = client.search_runs(experiment_ids="1")[0].info.run_id
mlflow.register_model(f"runs:/{run_id}/models", "IrisModel")

Successfully registered model 'IrisModel'.
2025/09/03 07:41:22 WARNING mlflow.tracking._model_registry.fluent: Run with id f3653b8c3a1c450c9cd4e40ba0460755 has no artifacts at artifact path 'models', registering model based on models:/m-b32f38c748464e75a1be03c3866092d5 instead
2025/09/03 07:41:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisModel, version 1
Created version '1' of model 'IrisModel'.


<ModelVersion: aliases=[], creation_timestamp=1756885282973, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1756885282973, metrics=None, model_id=None, name='IrisModel', params=None, run_id='f3653b8c3a1c450c9cd4e40ba0460755', run_link='', source='models:/m-b32f38c748464e75a1be03c3866092d5', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [30]:
# client.set_tag(run_id=run_id, key="env", value="Production")
client.set_aliase(run_id=run_id, key="env", value="Production")

# Or set a tag on a model version
# client.set_model_version_tag(
#     name="IrisModel",
#     version=1,
#     key="env",
#     value="Staging",
#     stage=None   # new param in your version
# )



AttributeError: 'MlflowClient' object has no attribute 'set_aliase'

In [17]:
client.search_registered_models()

[]

In [12]:
from mlflow.exceptions import MlflowException


In [16]:
try:
    client.search_registered_models()
    print(f"Access to the model registry is working")
except MlflowException:
    print(f"It's not possible to access the model registry")

Access to the model registry is working
